# Expedia Hotel Recommendations — Feature Engineering

Transform raw columns into rich, model-ready features that encode temporal, geographic, behavioural, and interaction signals.

---
## Table of Contents
1. [Setup & Data Loading](#1)
2. [Temporal Features](#2)
3. [Stay & Search Features](#3)
4. [Geographic Features](#4)
5. [User Behaviour Features](#5)
6. [Destination Aggregation Features](#6)
7. [Merge `destinations.csv`](#7)
8. [Missing Value Treatment](#8)
9. [Encoding Categorical Features](#9)
10. [Feature Scaling](#10)
11. [Feature Importance Preview](#11)
12. [Save Processed Dataset](#12)

---
## 1. Setup & Data Loading <a id='1'></a>

**What**: Import libraries and load a representative sample of `train.csv`.  
**Why**: All feature engineering steps are defined here and will later be wrapped into a reusable pipeline applied to the full dataset.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

# Paths
TRAIN_PATH   = 'expedia-hotel-recommendations/train.csv'
DEST_PATH    = 'expedia-hotel-recommendations/destinations.csv'
OUTPUT_PATH  = 'expedia-hotel-recommendations/train_features.parquet'
SAMPLE_SIZE  = 1000000   
TOTAL_ROWS   = 37670293
RANDOM_SEED  = 42

# Load raw data (random sample)
print(f"Sampling {SAMPLE_SIZE:,} random rows from {TOTAL_ROWS:,} total …")

np.random.seed(RANDOM_SEED)
skip_rows = np.random.choice(
    np.arange(1, TOTAL_ROWS + 1),       # all row indices (1-based, header=0)
    size=TOTAL_ROWS - SAMPLE_SIZE,      # how many rows to skip
    replace=False
)

df = pd.read_csv(TRAIN_PATH, skiprows=skip_rows)
print(f"✓ Raw data: {df.shape[0]:,} rows x {df.shape[1]} columns")
df.head(10)

Sampling 1,000,000 random rows from 37,670,293 total …
✓ Raw data: 1,000,000 rows x 24 columns


,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,3,2014-11-23,2014-11-28,2,0,1,14984,1,0,1,2,50,1457,21
1,2014-11-23 17:10:17,30,4,195,991,47725,NaN,1048,0,0,9,2015-06-26,2015-06-28,2,0,1,8803,1,0,1,3,151,69,82
2,2014-09-15 19:07:14,2,3,66,311,45008,NaN,1561,0,0,9,2014-09-19,2014-09-21,2,0,1,52565,7,0,1,2,50,674,68
3,2013-12-02 02:05:21,24,2,3,64,12576,NaN,2451,0,0,4,2014-02-28,2014-03-04,2,0,1,18489,6,0,1,6,105,29,8
4,2014-07-18 02:09:17,24,2,3,64,12576,NaN,2451,0,0,2,2014-08-27,2014-08-28,2,0,1,17823,1,0,1,6,105,770,99
5,2014-08-09 03:18:47,24,2,3,64,4777,NaN,2451,0,0,2,2014-08-30,2014-08-31,2,0,1,8746,1,0,1,6,105,29,95
6,2014-12-23 02:54:06,24,2,3,64,9448,NaN,2451,0,0,0,2015-02-05,2015-02-06,2,0,1,14041,6,0,1,6,105,35,8
7,2014-12-25 15:35:37,24,2,3,51,9527,NaN,2451,1,0,9,2015-02-06,2015-02-07,2,0,1,8785,1,0,1,6,105,35,85
8,2014-12-25 15:39:35,24,2,3,51,9527,NaN,2451,1,0,9,2015-02-06,2015-02-07,2,0,1,8785,1,0,1,6,105,35,85
9,2014-12-26 18:27:42,24,2,3,64,9448,NaN,2451,0,0,4,2015-02-05,2015-02-06,2,0,1,14041,6,0,1,6,105,35,64


---
## 2. Temporal Features <a id='2'></a>

Parse all date columns and extract calendar components + booking window metrics. ***When*** someone searches matters, for exmaple: 
- summer travellers book different hotels than winter ones,
- last-minute bookers differ from planners. 

In [3]:
def add_temporal_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Parse
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['srch_ci']   = pd.to_datetime(df['srch_ci'],  errors='coerce')
    df['srch_co']   = pd.to_datetime(df['srch_co'],  errors='coerce')

    # Search-event calendar
    df['search_year']      = df['date_time'].dt.year
    df['search_month']     = df['date_time'].dt.month
    df['search_weekofyear']= df['date_time'].dt.isocalendar().week.astype(int)
    df['search_weekday']   = df['date_time'].dt.dayofweek          # 0=Mon
    df['search_hour']      = df['date_time'].dt.hour
    df['search_quarter']   = df['date_time'].dt.quarter
    df['is_weekend_search']= df['search_weekday'].isin([5, 6]).astype(int)

    # Check-in calendar
    df['ci_month']         = df['srch_ci'].dt.month
    df['ci_weekday']       = df['srch_ci'].dt.dayofweek
    df['ci_quarter']       = df['srch_ci'].dt.quarter
    df['is_weekend_checkin']= df['ci_weekday'].isin([5, 6]).astype(int)

    # Seasonality (cyclical encoding avoids discontinuity at Dec to Jan)
    df['search_month_sin'] = np.sin(2 * np.pi * df['search_month'] / 12)
    df['search_month_cos'] = np.cos(2 * np.pi * df['search_month'] / 12)
    df['search_hour_sin']  = np.sin(2 * np.pi * df['search_hour']  / 24)
    df['search_hour_cos']  = np.cos(2 * np.pi * df['search_hour']  / 24)
    df['ci_month_sin']     = np.sin(2 * np.pi * df['ci_month']     / 12)
    df['ci_month_cos']     = np.cos(2 * np.pi * df['ci_month']     / 12)

    # Booking window & stay
    df['lead_time']        = (df['srch_ci'] - df['date_time']).dt.days
    df['stay_duration']    = (df['srch_co'] - df['srch_ci']).dt.days

    # Clip negatives (data anomalies)
    df['lead_time']        = df['lead_time'].clip(lower=0)
    df['stay_duration']    = df['stay_duration'].clip(lower=1)

    # Bucket lead time
    df['lead_time_bin'] = pd.cut(
        df['lead_time'].fillna(-1),
        bins=[-2, 0, 7, 30, 90, 180, 365, 9999],
        labels=['unknown','same_week','1_month','3_months','6_months','1_year','over_1_year']
    )

    return df

df = add_temporal_features(df)
new_cols = [c for c in df.columns if c not in pd.read_csv(TRAIN_PATH, nrows=1).columns]
print(f"✓ {len(new_cols)} temporal features added:")
for c in new_cols:
    print(f"   {c}")

✓ 20 temporal features added:
   search_year
   search_month
   search_weekofyear
   search_weekday
   search_hour
   search_quarter
   is_weekend_search
   ci_month
   ci_weekday
   ci_quarter
   is_weekend_checkin
   search_month_sin
   search_month_cos
   search_hour_sin
   search_hour_cos
   ci_month_sin
   ci_month_cos
   lead_time
   stay_duration
   lead_time_bin


---
## 3. Stay & Search Features <a id='3'></a>

Engineer features that describe the trip size and nature of the search. A solo last-minute 1-night stay and a family 2-week holiday land in very different hotel clusters. Combining raw counts into derived signals (total guests, room ratio, family flag) gives the model richer context.

In [ ]:
def add_search_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Guest & room ratios
    df['total_guests']        = df['srch_adults_cnt'] + df['srch_children_cnt']
    df['guests_per_room']     = df['total_guests'] / df['srch_rm_cnt'].replace(0, 1)
    df['adults_per_room']     = df['srch_adults_cnt'] / df['srch_rm_cnt'].replace(0, 1)
    df['children_per_room']   = df['srch_children_cnt'] / df['srch_rm_cnt'].replace(0, 1)

    # Trip type flags
    df['is_family']           = (df['srch_children_cnt'] > 0).astype(int)
    df['is_solo']             = ((df['srch_adults_cnt'] == 1) & (df['srch_children_cnt'] == 0)).astype(int)
    df['is_couple']           = ((df['srch_adults_cnt'] == 2) & (df['srch_children_cnt'] == 0)).astype(int)
    df['is_group']            = (df['srch_adults_cnt']  > 2).astype(int)
    df['is_multi_room']       = (df['srch_rm_cnt']      > 1).astype(int)

    # Stay classification
    df['is_long_stay']        = (df['stay_duration']    > 7).astype(int)
    df['is_short_stay']       = (df['stay_duration']   <= 3).astype(int)
    df['is_last_minute']      = (df['lead_time']        <= 3).astype(int)
    df['is_advance_booking']  = (df['lead_time']        >= 30).astype(int)

    # Log transforms: reduce skew for distance, lead_time.
    df['log_lead_time']       = np.log1p(df['lead_time'].clip(lower=0).fillna(0))
    df['log_stay_duration']   = np.log1p(df['stay_duration'].clip(lower=0).fillna(0))
    df['log_distance']        = np.log1p(df['orig_destination_distance'].fillna(0))

    return df

df = add_search_features(df)
print("✓ Stay & search features added")
df[['total_guests','guests_per_room','is_family','is_solo','is_couple',
    'is_group','is_last_minute','is_advance_booking',
    'log_lead_time','log_distance']].describe()

✓ Stay & search features added


,total_guests,guests_per_room,is_family,is_solo,is_couple,is_group,is_last_minute,is_advance_booking,log_lead_time,log_distance
count,1000000.0000,1000000.0000,1000000.0000,1000000.0000,1000000.0000,1000000.0000,1000000.0000,1000000.0000,1000000.0000,1000000.0000
mean,2.3569,2.1544,0.2099,0.1863,0.5099,0.1283,0.1545,0.4981,3.1746,4.2976
std,1.2313,0.9517,0.4072,0.3894,0.4999,0.3344,0.3614,0.5000,1.5012,3.4878
min,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
25%,2.0000,2.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2.1972,0.0000
50%,2.0000,2.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,3.4012,5.5183
75%,3.0000,2.0000,0.0000,0.0000,1.0000,0.0000,0.0000,1.0000,4.3041,7.4202
max,18.0000,14.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,6.3733,9.3970


---
## 4. Geographic Features <a id='4'></a>

Build location-based signals-proximity flags, same-country/continent matches, and cross features between user and hotel geography. ***Geographic distance*** between a user and a hotel is one of the strongest cluster predictors. Encoding it explicitly, along with same-region flags, gives the model direct access to this signal.

In [5]:
def add_geographic_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # User - Hotel proximity flags
    df['same_continent']      = (df['posa_continent']        == df['hotel_continent']).astype(int)
    df['same_country']        = (df['user_location_country'] == df['hotel_country']).astype(int)
    df['is_domestic']         = df['same_country']   # alias for readability

    # Distance bins (useful when distance is present)
    df['distance_bin'] = pd.cut(
        df['orig_destination_distance'].fillna(-1),
        bins=[-2, 0, 100, 500, 2000, 5000, 1e9],
        labels=['unknown', 'very_close', 'close', 'medium', 'far', 'very_far']
    )
    df['distance_known']      = df['orig_destination_distance'].notna().astype(int)

    # Composite geo keys (for later target encoding / aggregations)
    df['user_hotel_continent_pair'] = (
        df['posa_continent'].astype(str) + '_' + df['hotel_continent'].astype(str)
    )
    df['user_country_hotel_country'] = (
        df['user_location_country'].astype(str) + '_' + df['hotel_country'].astype(str)
    )
    df['destination_hotel_country'] = (
        df['srch_destination_id'].astype(str) + '_' + df['hotel_country'].astype(str)
    )

    return df

df = add_geographic_features(df)
print("✓ Geographic features added")
print("\nDomestic vs international searches:")
print(df['is_domestic'].value_counts(normalize=True).rename({1:'domestic', 0:'international'}).to_string())
print("\nDistance bin distribution:")
print(df['distance_bin'].value_counts().to_string())

✓ Geographic features added

Domestic vs international searches:
is_domestic
international   0.9987
domestic        0.0013

Distance bin distribution:
distance_bin
unknown       360440
medium        210912
close         149211
far           141217
very_far       75446
very_close     62774


---
## 5. User Behaviour Features <a id='5'></a>

Aggregate each user's historical search patterns into per-user statistics. A user who always books package deals, or always travels to the same continent, is very likely to repeat that behaviour. These look-back statistics carry strong predictive signal even for new sessions.

In [6]:
def add_user_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Per-user aggregates
    user_agg = df.groupby('user_id').agg(
        user_search_count       = ('user_id',          'count'),
        user_booking_rate       = ('is_booking',        'mean'),
        user_avg_lead_time      = ('lead_time',         'mean'),
        user_avg_stay           = ('stay_duration',     'mean'),
        user_mobile_rate        = ('is_mobile',         'mean'),
        user_package_rate       = ('is_package',        'mean'),
        user_avg_adults         = ('srch_adults_cnt',   'mean'),
        user_avg_children       = ('srch_children_cnt', 'mean'),
        user_unique_dest        = ('srch_destination_id','nunique'),
        user_unique_countries   = ('hotel_country',     'nunique'),
    ).reset_index()

    df = df.merge(user_agg, on='user_id', how='left')

    # User experience tiers
    df['user_is_power']   = (df['user_search_count'] > 10).astype(int)
    df['user_is_new']     = (df['user_search_count'] == 1).astype(int)
    df['user_is_booker']  = (df['user_booking_rate'] > 0).astype(int)   # ever booked

    # Log-transform heavy-tail count
    df['log_user_search_count'] = np.log1p(df['user_search_count'])

    return df

df = add_user_features(df)
print("✓ User behaviour features added")

user_cols = [c for c in df.columns if c.startswith('user_')]
print(f"\nNew user columns: {user_cols}")
df[user_cols].describe()

✓ User behaviour features added

New user columns: ['user_location_country', 'user_location_region', 'user_location_city', 'user_id', 'user_hotel_continent_pair', 'user_country_hotel_country', 'user_search_count', 'user_booking_rate', 'user_avg_lead_time', 'user_avg_stay', 'user_mobile_rate', 'user_package_rate', 'user_avg_adults', 'user_avg_children', 'user_unique_dest', 'user_unique_countries', 'user_is_power', 'user_is_new', 'user_is_booker']


,user_location_country,user_location_region,user_location_city,user_id,user_search_count,user_booking_rate,user_avg_lead_time,user_avg_stay,user_mobile_rate,user_package_rate,user_avg_adults,user_avg_children,user_unique_dest,user_unique_countries,user_is_power,user_is_new,user_is_booker
count,1000000.0000,1000000.0000,1000000.0000,1000000.0000,1000000.0000,1000000.0000,999545.0000,999545.0000,1000000.0000,1000000.0000,1000000.0000,1000000.0000,1000000.0000,1000000.0000,1000000.0000,1000000.0000,1000000.0000
mean,86.0793,308.1245,27727.6364,604545.7937,3.5316,0.0798,53.7965,3.3774,0.1350,0.2491,2.0248,0.3320,2.8007,1.8556,0.0315,0.2689,0.2041
std,59.2612,208.5128,16783.7180,350575.3750,2.8171,0.2024,55.2108,2.4046,0.3071,0.3703,0.7308,0.6341,2.1412,1.3085,0.1747,0.4434,0.4031
min,0.0000,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,1.0000,1.0000,0.0000,0.0000,0.0000
25%,66.0000,174.0000,12907.0000,298524.5000,1.0000,0.0000,14.2500,2.0000,0.0000,0.0000,1.7143,0.0000,1.0000,1.0000,0.0000,0.0000,0.0000
50%,66.0000,314.0000,27655.0000,604039.5000,3.0000,0.0000,36.1429,3.0000,0.0000,0.0000,2.0000,0.0000,2.0000,1.0000,0.0000,0.0000,0.0000
75%,70.0000,385.0000,42328.0000,910727.2500,5.0000,0.0000,74.9091,4.1250,0.0000,0.5000,2.0000,0.4286,4.0000,2.0000,0.0000,1.0000,0.0000
max,239.0000,1027.0000,56507.0000,1198780.0000,22.0000,1.0000,585.0000,213.0000,1.0000,1.0000,9.0000,9.0000,18.0000,15.0000,1.0000,1.0000,1.0000


---
## 6. Destination Aggregation Features <a id='6'></a>

Compute popularity and booking statistics per destination and per hotel market. Some destinations are overwhelmingly associated with specific hotel clusters. Encoding the ***popularity of each cluster per destination*** is one of the most powerful signals in this dataset.

In [7]:
def add_destination_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Destination-level stats
    dest_agg = df.groupby('srch_destination_id').agg(
        dest_search_count   = ('srch_destination_id', 'count'),
        dest_booking_rate   = ('is_booking',           'mean'),
        dest_avg_adults     = ('srch_adults_cnt',      'mean'),
        dest_avg_stay       = ('stay_duration',        'mean'),
        dest_avg_lead_time  = ('lead_time',            'mean'),
        dest_package_rate   = ('is_package',           'mean'),
    ).reset_index()
    df = df.merge(dest_agg, on='srch_destination_id', how='left')

    # Most popular cluster per destination (only from bookings)
    # Using only booking rows avoids click noise
    bookings = df[df['is_booking'] == 1]

    dest_top_cluster = (
        bookings.groupby(['srch_destination_id', 'hotel_cluster'])
        .size()
        .reset_index(name='cnt_cluster')
        .sort_values('cnt_cluster', ascending=False)
        .drop_duplicates(subset='srch_destination_id')
        .rename(columns={'hotel_cluster': 'dest_top_cluster'})
        [['srch_destination_id', 'dest_top_cluster']]
    )
    df = df.merge(dest_top_cluster, on='srch_destination_id', how='left')

    # Hotel market stats
    market_agg = df.groupby('hotel_market').agg(
        market_search_count = ('hotel_market', 'count'),
        market_booking_rate = ('is_booking',   'mean'),
    ).reset_index()
    df = df.merge(market_agg, on='hotel_market', how='left')

    # Log transforms
    df['log_dest_search_count']   = np.log1p(df['dest_search_count'])
    df['log_market_search_count'] = np.log1p(df['market_search_count'])

    return df

df = add_destination_features(df)
print("✓ Destination aggregation features added")

dest_cols = [c for c in df.columns if c.startswith('dest_') or c.startswith('market_')]
print(f"\nNew destination/market columns: {dest_cols}")
df[dest_cols].describe()

✓ Destination aggregation features added

New destination/market columns: ['dest_search_count', 'dest_booking_rate', 'dest_avg_adults', 'dest_avg_stay', 'dest_avg_lead_time', 'dest_package_rate', 'dest_top_cluster', 'market_search_count', 'market_booking_rate']


,dest_search_count,dest_booking_rate,dest_avg_adults,dest_avg_stay,dest_avg_lead_time,dest_package_rate,dest_top_cluster,market_search_count,market_booking_rate
count,1000000.0000,1000000.0000,1000000.0000,999954.0000,999954.0000,1000000.0000,962229.0000,1000000.0000,1000000.0000
mean,4643.9477,0.0798,2.0248,3.3771,53.7956,0.2491,48.6067,9668.7806,0.0798
std,7995.3802,0.0644,0.2004,1.2507,21.7366,0.2153,30.5431,12804.8901,0.0386
min,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,1.0000,0.0000
25%,202.0000,0.0493,1.9347,2.4888,38.7200,0.0645,22.0000,944.0000,0.0565
50%,1163.0000,0.0670,2.0305,3.3142,54.3583,0.2199,50.0000,4159.0000,0.0719
75%,5058.0000,0.0952,2.1235,4.0000,66.3530,0.3636,68.0000,12828.0000,0.0990
max,35348.0000,1.0000,9.0000,38.0000,489.0000,1.0000,99.0000,47001.0000,0.6667


---
## 7. Merge `destinations.csv` <a id='7'></a>

Join the `destinations.csv` file which contains 149 latent features (PCA components) describing each destination.  These latent features capture the ***character*** of each destination (beach resort, city, ski resort…) and are highly predictive of hotel cluster choice.

In [9]:
dest_meta = pd.read_csv(DEST_PATH)
print(f"destinations.csv shape: {dest_meta.shape}")
print(f"Columns: {list(dest_meta.columns[:5])} … ({dest_meta.shape[1]} total)")
dest_meta.head(5)

destinations.csv shape: (62106, 150)
Columns: ['srch_destination_id', 'd1', 'd2', 'd3', 'd4'] … (150 total)


,srch_destination_id,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20,d21,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31,d32,d33,d34,d35,d36,d37,d38,d39,d40,d41,d42,d43,d44,d45,d46,d47,d48,d49,d50,d51,d52,d53,d54,d55,d56,d57,d58,d59,d60,d61,d62,d63,d64,d65,d66,d67,d68,d69,d70,d71,d72,d73,d74,d75,d76,d77,d78,d79,d80,d81,d82,d83,d84,d85,d86,d87,d88,d89,d90,d91,d92,d93,d94,d95,d96,d97,d98,d99,d100,d101,d102,d103,d104,d105,d106,d107,d108,d109,d110,d111,d112,d113,d114,d115,d116,d117,d118,d119,d120,d121,d122,d123,d124,d125,d126,d127,d128,d129,d130,d131,d132,d133,d134,d135,d136,d137,d138,d139,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149
0,0,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-1.8976,-2.1987,-2.1987,-1.8976,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-1.8976,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-1.8976,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-1.8976,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-1.8976,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-1.8976,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-1.8976,-2.1987,-2.1987,-2.1987,-1.8976,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987,-2.1987
1,1,-2.1817,-2.1817,-2.1817,-2.0826,-2.1817,-2.1650,-2.1817,-2.1817,-2.0316,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.0438,-2.1817,-2.1817,-2.1817,-2.1650,-2.1817,-2.1335,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1335,-2.1817,-2.1817,-2.1817,-2.1817,-2.0826,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1650,-2.1650,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.0826,-2.1817,-2.1335,-2.1817,-2.1817,-2.1817,-2.1817,-2.0826,-2.1817,-2.1817,-2.1817,-2.1817,-2.1650,-2.1817,-2.1817,-2.1817,-2.1817,-2.1490,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1335,-2.1817,-2.1817,-2.1817,-2.1817,-2.1817,-2.1335,-2.1817,-2.1817,-2.1817,-2.1490,-2.1817,-2.1817,-2.1817,-2.1817,-2.1490,-2.1817,-2.1817,-2.1650,-2.1817,-2.1650,-2.1817,-2.1817,-2.1650,-2.1817,-2.1817,-2.1817,-2.1817
2,2,-2.1835,-2.2242,-2.2242,-2.1896,-2.1058,-2.0754,-2.2242,-2.1185,-2.1404,-2.2242,-2.2099,-2.2242,-2.1107,-2.1860,-2.2242,-2.1245,-2.2242,-2.1565,-2.2242,-2.2242,-2.2242,-2.2242,-2.2242,-2.2242,-2.2242,-2.2242,-2.2242,-2.2242,-2.1533,-2.1860,-2.2242,-2.2242,-2.2242,-2.1282,-2.1583,-2.0597,-1.9103,-2.2242,-2.1231,-2.2242,-2.1540,-2.1973,-1.9455,-2.2242,-2.2242,-2.1973,-2.1669,-2.1920,-1.9038,-2.2242,-2.1896,-2.2242,-2.0831,-2.2242,-2.0701,-2.1577,-2.1964,-2.0698,-2.2242,-2.2242,-2.0730,-2.2242,-2.2242,-2.2242,-2.2242,-2.1404,-2.2242,-2.2242,-2.0966,-2.2242,-2.1875,-2.2242,-2.2242,-2.0987,-2.0164,-2.2242,-2.1261,-2.1896,-2.0083,-2.0858,-2.2242,-2.2242,-2.2242,-2.2184,-2.2193,-2.2242,-2.2242,-2.1605,-2.2242,-2.2242,-2.0768,-2.2242,-1.9751,-2.1661,-2.2242,-2.2242,-2.2242,-2.2242,-2.2242,-2.1637,-2.2242,-2.0724,-2.1003,-2.2242,-2.2035,-2.2242,-2.2242,-2.0944,-2.2242,-2.1404,-2.2242,-2.1647,-2.2242,-2.1550,-2.1973,-2.2242,-2.1973,-2.2242,-2

In [10]:
# Optionally reduce dimensionality with PCA to keep top N components
# Full 149 components can be used; here we keep all of them
dest_feature_cols = [c for c in dest_meta.columns if c != 'srch_destination_id']
print(f"Merging {len(dest_feature_cols)} destination latent features …")

df = df.merge(
    dest_meta.rename(columns={'srch_destination_id': 'srch_destination_id'}),
    on='srch_destination_id',
    how='left'
)

coverage = df[dest_feature_cols[0]].notna().mean() * 100
print(f"✓ Merge complete — destination features cover {coverage:.1f}% of rows")
print(f"  Dataset now: {df.shape[0]:,} rows × {df.shape[1]} columns")

Merging 149 destination latent features …
✓ Merge complete — destination features cover 99.6% of rows
  Dataset now: 1,000,000 rows × 242 columns


---
## 8. Missing Value Treatment <a id='8'></a>

Apply a systematic strategy to handle every remaining NaN. Most tree-based models can handle NaN internally, but scikit-learn and neural networks cannot. Explicit imputation also ensures reproducibility and consistency between train and test sets.

In [ ]:
def treat_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 1. Distance — already flagged; fill raw with median
    dist_median = df['orig_destination_distance'].median()
    df['orig_destination_distance'] = df['orig_destination_distance'].fillna(dist_median)

    # 2. lead_time / stay_duration — fill with median
    df['lead_time']     = df['lead_time'].fillna(df['lead_time'].median())
    df['stay_duration'] = df['stay_duration'].fillna(df['stay_duration'].median())

    # 3. User aggregates — new users have no history → fill with 0 or global mean
    user_agg_cols = [c for c in df.columns if c.startswith('user_') and c != 'user_id']
    for col in user_agg_cols:
        df[col] = df[col].fillna(0)

    # 4. Destination aggregates — unknown destinations → fill with global median
    dest_agg_cols = [c for c in df.columns if c.startswith('dest_') or c.startswith('market_')]
    for col in dest_agg_cols:
        fill = df[col].median() if df[col].dtype != object else 'unknown'
        df[col] = df[col].fillna(fill)

    # 5. Destination latent features (d1–d149) → fill with 0 (mean after PCA)
    d_cols = [c for c in df.columns if c.startswith('d') and c[1:].isdigit()]
    df[d_cols] = df[d_cols].fillna(0)

    # 6. Categorical bins → add 'unknown' category explicitly
    for col in ['lead_time_bin', 'distance_bin']:
        if col in df.columns:
            df[col] = df[col].astype(str).replace('nan', 'unknown')

    return df

df = treat_missing_values(df)

# Verify
remaining_na = df.isnull().sum()
remaining_na = remaining_na[remaining_na > 0]
if remaining_na.empty:
    print("✓ No missing values remaining!")
else:
    print(f"!!! Still missing in {len(remaining_na)} columns:")
    print(remaining_na.to_string())

---
## 9. Encoding Categorical Features <a id='9'></a>

Convert string/object and high-cardinality integer columns into numeric representations. Models require numbers. We use ***target encoding*** (mean of `hotel_cluster`) for high-cardinality columns instead of one-hot encoding, to avoid the curse of dimensionality and preserve the signal in rare categories.

In [ ]:
def target_encode(df: pd.DataFrame, cols: list, target: str = 'hotel_cluster',
                  smoothing: float = 10.0) -> pd.DataFrame:
    """
    Smoothed target encoding:
        encoded = (n_i * mean_i + k * global_mean) / (n_i + k)
    where k = smoothing factor.  Reduces noise for rare categories.
    """
    df = df.copy()
    global_mean = df[target].mean()

    for col in cols:
        stats = df.groupby(col)[target].agg(['mean', 'count'])
        smoothed = (stats['count'] * stats['mean'] + smoothing * global_mean) \
                   / (stats['count'] + smoothing)
        df[f'{col}_te'] = df[col].map(smoothed).fillna(global_mean)

    return df


# High-cardinality columns → target encode
HIGH_CARD_COLS = [
    'user_location_country',
    'user_location_region',
    'hotel_country',
    'hotel_market',
    'srch_destination_id',
    'site_name',
    'channel',
]
df = target_encode(df, HIGH_CARD_COLS)
te_cols = [f'{c}_te' for c in HIGH_CARD_COLS]
print(f"✓ Target-encoded {len(HIGH_CARD_COLS)} columns:")
for c in te_cols:
    print(f"   {c}")

# Low-cardinality bins → label encode
BIN_COLS = ['lead_time_bin', 'distance_bin']
le = LabelEncoder()
for col in BIN_COLS:
    if col in df.columns:
        df[f'{col}_enc'] = le.fit_transform(df[col].astype(str))
        print(f"✓ Label-encoded: {col}")

# Drop original object columns (keep encoded versions)
drop_cols = ['date_time', 'srch_ci', 'srch_co',
             'user_hotel_continent_pair', 'user_country_hotel_country',
             'destination_hotel_country', 'lead_time_bin', 'distance_bin']
df.drop(columns=[c for c in drop_cols if c in df.columns], inplace=True)
print(f"\n✓ Dataset shape after encoding: {df.shape[0]:,} rows × {df.shape[1]} columns")

---
## 10. Feature Scaling <a id='10'></a>

Standardise continuous features to zero mean / unit variance. Tree-based models (LightGBM, XGBoost) are scale-invariant, but linear models, SVMs, and neural networks are sensitive to feature magnitude. We apply scaling here for completeness and keep unscaled columns available.

In [ ]:
SCALE_COLS = [
    'orig_destination_distance', 'log_distance',
    'lead_time',   'log_lead_time',
    'stay_duration','log_stay_duration',
    'total_guests', 'guests_per_room',
    'user_search_count', 'log_user_search_count',
    'user_avg_lead_time', 'user_avg_stay',
    'dest_search_count',  'log_dest_search_count',
    'market_search_count','log_market_search_count',
]
SCALE_COLS = [c for c in SCALE_COLS if c in df.columns]

scaler   = StandardScaler()
scaled   = scaler.fit_transform(df[SCALE_COLS])
scaled_df = pd.DataFrame(scaled, columns=[f'{c}_scaled' for c in SCALE_COLS],
                         index=df.index)
df = pd.concat([df, scaled_df], axis=1)

print(f"✓ Scaled {len(SCALE_COLS)} continuous features")
print(f"  Dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

df[[f'{c}_scaled' for c in SCALE_COLS[:4]]].describe().round(3)

---
## 11. Feature Importance Preview <a id='11'></a>

Train a shallow Random Forest on the engineered features and rank them by importance. A quick importance check validates that the new features actually carry signal before spending compute on full model training. It also helps drop noise columns early.

In [ ]:
# Select only numeric, non-target columns
TARGET   = 'hotel_cluster'
DROP_FOR_MODEL = ['user_id', TARGET]

model_cols = [
    c for c in df.select_dtypes(include=[np.number]).columns
    if c not in DROP_FOR_MODEL and not c.endswith('_scaled')  # avoid duplicate info
]

X = df[model_cols].fillna(0)
y = df[TARGET]

print(f"Training shallow Random Forest on {len(model_cols)} features …")
rf = RandomForestClassifier(n_estimators=50, max_depth=8, n_jobs=-1, random_state=42)
rf.fit(X, y)

# Plot top-30 features
importances = pd.Series(rf.feature_importances_, index=model_cols).sort_values(ascending=False)
top30 = importances.head(30)

plt.figure(figsize=(12, 10))
plt.barh(top30.index[::-1], top30.values[::-1], color='steelblue', alpha=0.85)
plt.xlabel('Feature Importance (Gini)', fontsize=12)
plt.title('Top 30 Feature Importances — Random Forest Preview', fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("\nTop 15 features:")
print(top30.head(15).to_string())

---
## 12. Save Processed Dataset <a id='12'></a>

Persist the fully engineered feature matrix to disk in Parquet format. Parquet is column-oriented, compressed, and retains dtype information. It saves hours of re-running this pipeline when you move to model training. Load it with `pd.read_parquet()`.

In [ ]:
# Keep only numeric columns for the model file
final_cols = [c for c in df.select_dtypes(include=[np.number]).columns]
df_out = df[final_cols].copy()

# Save
df_out.to_parquet(OUTPUT_PATH, index=False, compression='snappy')

size_mb = pd.io.common.get_filepath_or_buffer(OUTPUT_PATH)
import os
size_mb = os.path.getsize(OUTPUT_PATH) / 1024**2

print(f"  Saved --> {OUTPUT_PATH}")
print(f"  Rows      : {df_out.shape[0]:,}")
print(f"  Columns   : {df_out.shape[1]}")
print(f"  File size : {size_mb:.1f} MB")

# Quick reload test
df_check = pd.read_parquet(OUTPUT_PATH)
print(f"\n✓ Reload test passed — shape: {df_check.shape}")

In [ ]:
# Final summary of all engineered features
raw_cols = set(pd.read_csv(TRAIN_PATH, nrows=1).columns)
new_features = [c for c in df_out.columns if c not in raw_cols]

print("=" * 70)
print("FEATURE ENGINEERING SUMMARY")
print("=" * 70)
print(f"  Original columns   : {len(raw_cols)}")
print(f"  Engineered columns : {len(new_features)}")
print(f"  Total in output    : {df_out.shape[1]}")
print(f"  Rows saved         : {df_out.shape[0]:,}")
print("=" * 70)

groups = {
    "Temporal"       : [c for c in new_features if any(k in c for k in ['search_', 'ci_', 'lead', 'stay', 'season', 'weekend', 'quarter', 'hour', 'month', 'week'])],
    "Search / Trip"  : [c for c in new_features if any(k in c for k in ['guest', 'room', 'solo', 'couple', 'family', 'group', 'multi', 'long_stay', 'short_stay', 'last_minute', 'advance', 'log_lead', 'log_stay', 'log_dist'])],
    "Geographic"     : [c for c in new_features if any(k in c for k in ['same_', 'domestic', 'distance_bin', 'distance_known'])],
    "User behaviour" : [c for c in new_features if c.startswith('user_')],
    "Destination"    : [c for c in new_features if c.startswith('dest_') or c.startswith('market_')],
    "Target encoded" : [c for c in new_features if c.endswith('_te')],
    "Scaled"         : [c for c in new_features if c.endswith('_scaled')],
    "Destination PCA": [c for c in new_features if c.startswith('d') and c[1:].isdigit()],
}

for grp, cols in groups.items():
    print(f"\n  {grp} ({len(cols)} features):")
    for c in cols[:6]:
        print(f"    • {c}")
    if len(cols) > 6:
        print(f"    … and {len(cols)-6} more")